### COLETA DE DADOS

In [1]:
# CARREGANDO DADOS
from ucimlrepo import fetch_ucirepo 

# importando dataset
dataset = fetch_ucirepo(id=848) # 61068 registros e 20 atributos
  
# coletando as informações
data_frame = dataset.data.original

### PRÉ-PROCESSAMENTO DE DADOS

In [2]:
# TRATANDO DADOS
import pandas
from sklearn.preprocessing import LabelEncoder

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  61069
VALORES DUPLICADOS:  146
PÓS TRATAMENTO:  60923


### DIVISÃO DE DADOS

In [3]:
print(data_frame.columns)

Index(['class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color', 'stem-height',
       'stem-width', 'stem-color', 'has-ring', 'ring-type', 'habitat',
       'season'],
      dtype='object')


In [4]:
# DIVIDINDO O DATASET TRATADO
import numpy
from sklearn.model_selection import train_test_split

atributos = data_frame.drop(["class"], axis=1)
respostas = data_frame[["class"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)

# convertendo de (N, 1) para (N,)
r_treino = numpy.ravel(r_treino)
r_teste = numpy.ravel(r_teste)

In [5]:
print("TAMANHO DO DATASET TRATADO: ", len(data_frame))
print("CONJUNTO DE TREINO: ", a_treino.shape, " e ", r_treino.shape)
print("CONJUNTO DE TESTE: ", a_teste.shape, " e ", r_teste.shape)

TAMANHO DO DATASET TRATADO:  60923
CONJUNTO DE TREINO:  (42646, 14)  e  (42646,)
CONJUNTO DE TESTE:  (18277, 14)  e  (18277,)


### TREINAMENTO E AVALIAÇÃO DO MODELO

In [6]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "A", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "adam", "max_iter": 700},
    {"id": "B", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "adam", "max_iter": 600},
    {"id": "C", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 700},
    {"id": "D", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 600},
]

In [7]:
# APLICANDO MODELO
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando classificador
    classificador = MLPClassifier(
        hidden_layer_sizes=parametros["hidden_layer_sizes"],
        activation=parametros["activation"],
        solver=parametros["solver"],
        max_iter=parametros["max_iter"],
        random_state=42
    )

    # treinando o modelo
    classificador.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = classificador.predict(a_teste)

    # calculando métricas
    acuracia = accuracy_score(r_teste, r_previsao)
    f1 = f1_score(r_teste, r_previsao, average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia, f1])

c:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


KeyboardInterrupt: 

In [9]:
# EXIBINDO RESULTADOS
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "Acurácia", "F1-score"])
print(tabela_resultados)

  Identificador  Acurácia  F1-score
0             A  0.998468  0.998468
1             B  0.990918  0.990916
2             C  0.998249  0.998249
3             D  0.985282  0.985285
